# F27. SQL을 이용해 DB와 대화해보자

- 파이썬에서 DB에 접근하기 위한 방법 중 가장 많이 사용하게 되는 것은 Python Database API.
- Python DB-API는 여러 DB에 접근할 수 있는 표준 API이며, 크게 3 가지 작업을 한다.
    - DB 연결
    - SQL 쿼리 실행
    - DB 연결 닫기

- 기본적으로 PEP249 인터페이스 따름

![image](https://user-images.githubusercontent.com/80008411/144697537-f676e03c-32f6-4321-86e2-4ba8435f2768.png)

## 목표

1. 기본 SQL 쿼리 알기
2. 파이썬으로 DB를 연결해 SQL 쿼리 시도하기


# SQLite

- SQLite는 서버가 필요 없이 DB의 파일에 기초해 DB 처리를 구현한 임베디드 SQL DB 엔진
- 별도 설치 없이 쉽고 편리하게 사용 가능
- 파이썬 2.5 이상 버전에 기본 내장


In [2]:
import os
import sqlite3

In [3]:
db_path = os.getenv('HOME') + '/mydb.db'
conn = sqlite3.connect(db_path)
print(conn)

In [5]:
# sql 쿼리를 수행하고 결과를 얻는데 사용하는 객체
# insert 같이 DB에만 적용되는 명령어를 사용할 경우 cursor가 필요하지 않다
# 그러나 select 같이 db의 데이터를 불러올 때는 sql 쿼리 수행 결과에 접근하기 위해 cursor가 꼭 필요함

c = conn.cursor()
print(c)

In [7]:
# stocks라는 이름을 테이블 생성
c.execute('CREATE TABLE IF NOT EXISTS stocks(date text, trans text, symbol text, qty real, price real)')

# stocks 테이블에 데이터를 하나 인서트
c.execute("INSERT INTO stocks VALUES ('20200701', 'TEST', 'AIFFEL', 1, 10000)")

# 방금 인서트한 데이터 조회
c.execute('SELECT * FROM stocks')

# 조회 내역을 커서로 가져와서 출력
print(c.fetchone())

('20200701', 'TEST', 'AIFFEL', 1.0, 10000.0)


In [8]:
# commit을 통해 db 업데이트
# commit을 하지 않으면 db에 반영되지 않음
c.close()  # 먼저 커서를 닫고
conn.close()  # db 연결 닫기 & db에 데이터 변경 반영

### - 트랜잭션

- 쪼개질 수 없는 업무 처리 단위
- 예를 들어 ATM 계좌이체 시 출금 계좌와 입금 계좌 양쪽에서 금액이 빠지고 더해지는 과정이 쪼개져서는 안 됨.
- 또 온라인 쇼핑에서 결제 완료와 상품 배송 처리가 쪼개지면 안 됨.
- ACID: db 트랜잭션이 안전하게 수행된다는 것을 보장하기 위한 성질을 가리키는 약어
    - 원자성 Atomicity: 트랜잭션에 속한 작업들이 모두 수행되었는지 보장하는 능력. 즉 중간까지만 실행되고 실패하는 일은 없도록 하는 것
    - 일관성 Consistency: 트랜잭션이 실행을 성공적으로 완료하면 언제나 일관성 있는 데이터베이스 상태로 유지
    - 고립성 Isolation: 트랜잭션 수행 시 다른 트랜잭션의 연산 작업이 끼어들지 못하게 보장
    - 지속성 Durability: 성공적으로 수행된 트랜잭션은 영원히 반영되어야 함
- 부분 작업들 여러 개가 모여진 이러한 트랜잭션을 처리하기 위해 db는 다음과 같은 기술을 제공한다.
    - 롤백: 부분 작업이 실패하면 트랜잭션 실행 전으로 되돌림
    - 커밋: 모든 부분작업이 정상 완료되면 변경사항을 한꺼번에 db에 반영


In [13]:
import sqlite3
import os
db_path = os.getenv('HOME')+'/mydb.db'

conn = sqlite3.connect(db_path)
c = conn.cursor()

#- ! 재실행 시 테이블이 존재할 수 있으므로 아래처럼 해당 테이블들을 모두 지워줍니다.
c.execute("DROP TABLE IF EXISTS 도서대출내역")
c.execute("DROP TABLE IF EXISTS 도서대출내역2")
c.execute("DROP TABLE IF EXISTS 대출내역")
c.execute("DROP TABLE IF EXISTS 도서명")


#----- 1st table : 도서대출내역 -----#
c.execute("CREATE TABLE IF NOT EXISTS 도서대출내역 (ID varchar, 이름 varchar, 도서ID varchar, 대출일 varchar, 반납일 varchar)")
#- 생성(create)문 : 테이블명, 변수명, 변수타입을 지정

data = [('101','문강태','aaa','2020-06-01','2020-06-05'),
             ('101','문강태','ccc','2020-06-20','2020-06-25'),
             ('102','고문영','bbb','2020-06-01',None),
             ('102','고문영','ddd','2020-06-08',None),
             ('103','문상태','ccc','2020-06-01','2020-06-05'),
             ('104','강기둥',None,None,None)]
#- 입력할 데이터를 그대로 입력 (변수명 순서 기준대로)

c.executemany('INSERT INTO 도서대출내역 VALUES (?,?,?,?,?)', data)
#- 입력할 데이터를 실제 테이블에 insert하기
#-----------------------------------------------#


#----- 2nd table : 도서대출내역2 -----#
c.execute("CREATE TABLE IF NOT EXISTS 도서대출내역2 (ID varchar, 이름 varchar, 대출년월 varchar, 대출일수 varchar)")

data = [('101','문강태','2020-06','20일'),
             ('102','고문영','2020-06','10일'),
             ('103','문상태','2020-06','8일'),
             ('104','강기둥','2020-06','3일')]
c.executemany('INSERT INTO 도서대출내역2 VALUES (?,?,?,?)', data)
#--------------------------------------------------#


#----- 3rd table : 대출내역 -----#
c.execute("CREATE TABLE IF NOT EXISTS 대출내역 (ID varchar, 이름 varchar, 도서ID varchar)")

data = [('101','문강태','aaa'),
             ('102','고문영','bbb'),
             ('102','고문영','fff'),
             ('103','문상태','ccc'),
             ('104','강기둥',None)]
c.executemany('INSERT INTO 대출내역 VALUES (?,?,?)', data)
#-----------------------------------------#


#----- 4th table : 도서명 -----#
c.execute("CREATE TABLE IF NOT EXISTS 도서명 (도서ID varchar, 도서명 varchar)")

data = [('aaa','악몽을 먹고 자란 소년'),
             ('bbb','좀비아이'),
             ('ccc','공룡백과사전'),
             ('ddd','빨간구두'),
             ('eee','잠자는 숲속의 미녀')]

c.executemany('INSERT INTO 도서명 VALUES (?,?)', data)
#--------------------------------------#

conn.commit()
conn.close()

OperationalError: ignored

In [10]:
#----- 4th table : 도서명 -----#

# [1]
c.execute("CREATE TABLE IF NOT EXISTS 도서명 (도서ID varchar, 도서명 varchar)")

# [2]
data = [('aaa','악몽을 먹고 자란 소년'),
             ('bbb','좀비아이'),
             ('ccc','공룡백과사전'),
             ('ddd','빨간구두'),
             ('eee','잠자는 숲속의 미녀')]

# [3]
# 한꺼번에 여러 데이터를 처리
# 입력 데이터를 list로 관리할 경우 유용
c.executemany('INSERT INTO 도서명 VALUES (?,?)', data)
#--------------------------------------#

In [11]:
# DDL문으로 테이블 생성
conn = sqlite3.connect(db_path)
c = conn.cursor()

for row in c.execute('SELECT * FROM 도서명'):
  print(row)